## Inputs
---

In [1]:
# reading data and working with arrays
import h5py, nrrd
import numpy as np

# data paths
data_path = '/groups/scicompsoft/home/fleishmang/rnr-exm-work/test_data/mouse_pair5.h5'
hdf5_file = h5py.File(data_path, 'r')
fix = hdf5_file['/fixed'][...]
mov = hdf5_file['/move'][...]

# assumed spacings (there are no provided spacings, this is a problem)
fix_spacing = np.array([0.4, 0.1625, 0.1625])
mov_spacing = np.array([0.4, 0.1625, 0.1625])

# # write some channels
# nrrd.write('./fix.nrrd', fix.transpose(2,1,0), compression_level=2)
# nrrd.write('./mov.nrrd', mov.transpose(2,1,0), compression_level=2)

## Alignment
---

### affine

In [2]:
# alignment functions
from bigstream.align import alignment_pipeline
from bigstream.transform import apply_transform

# define alignment steps
common_kwargs = {
    'alignment_spacing':0.5,
    'shrink_factors':(8,4,2,1),
    'smooth_sigmas':(4.,4.,2.,1.),
    'optimizer_args':{
        'learningRate':0.25,
        'minStep':0.,
        'numberOfIterations':400,
    },
}

steps = [('affine', common_kwargs),]

IBM Spectrum LSF 10.1.0.0 build 601088, Apr 15 2022
Copyright International Business Machines Corp. 1992, 2016.
US Government Users Restricted Rights - Use, duplication or disclosure restricted by GSA ADP Schedule Contract with IBM Corp.

  binary type: linux3.10-glibc2.17-x86_64


In [3]:
%%time

# align
affine = alignment_pipeline(
    fix, mov,
    fix_spacing, mov_spacing,
    steps,
)

LEVEL:  0  ITERATION:  0  METRIC:  -0.15403506752833043
LEVEL:  0  ITERATION:  1  METRIC:  -0.15693137226072473
LEVEL:  0  ITERATION:  2  METRIC:  -0.15949087427236383
LEVEL:  0  ITERATION:  3  METRIC:  -0.16176669621525913
LEVEL:  0  ITERATION:  4  METRIC:  -0.16386176317760648
LEVEL:  0  ITERATION:  5  METRIC:  -0.16581494653272552
LEVEL:  0  ITERATION:  6  METRIC:  -0.1770689594704269
LEVEL:  0  ITERATION:  7  METRIC:  -0.17169448536677465
LEVEL:  0  ITERATION:  8  METRIC:  -0.17267725380670076
LEVEL:  0  ITERATION:  9  METRIC:  -0.1729883417408631
LEVEL:  0  ITERATION:  10  METRIC:  -0.17316563188325804
LEVEL:  0  ITERATION:  11  METRIC:  -0.1735076319106144
LEVEL:  0  ITERATION:  12  METRIC:  -0.17375002608859533
LEVEL:  0  ITERATION:  13  METRIC:  -0.17383130480438827
LEVEL:  1  ITERATION:  0  METRIC:  -0.17223082107937146
LEVEL:  1  ITERATION:  1  METRIC:  -0.17325930696877231
LEVEL:  2  ITERATION:  0  METRIC:  -0.12395195367660458
LEVEL:  2  ITERATION:  1  METRIC:  -0.125014131

In [4]:
# apply transform and save results
affine_aligned = apply_transform(
    fix, mov,
    fix_spacing, mov_spacing,
    transform_list=[affine,],
)

# write results
np.savetxt('affine.mat', affine)
nrrd.write('./affine.nrrd', affine_aligned.transpose(2,1,0), compression_level=2)

# load precomputed results
affine = np.loadtxt('./affine.mat')

In [5]:
from bigstream.utility import matrix_to_displacement_field

# load affine, convert to displacement, convert to voxel units
affine = np.loadtxt('./affine.mat')
deform = matrix_to_displacement_field(affine, fix.shape, spacing=fix_spacing)
deform = (deform / fix_spacing).astype(np.float32)

# save as h5 file
with h5py.File('../submission/mouse_test.h5', 'a') as hf:
    hf.create_dataset('pair5', data=deform, compression='gzip')